# Load libraries

In [1]:
import numpy as np
import tensorflow as tf

c:\users\silver\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Define Model (Estimator)

In [3]:
def model_fn(features, labels, mode):
    # Define Model parameters
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    
    # Define Model structure  y = f(x)
    y = tf.matmul(features['x'], W) + b
    
    # Define Model Loss
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=y))
    
    # Define Optimizer and step
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = tf.group(optimizer.minimize(cross_entropy),
                     tf.assign_add(global_step, 1))
    
    # Define other evaluation metrics 
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(tf.argmax(labels,1), tf.argmax(y,1))
    }
    return tf.estimator.EstimatorSpec(
              mode=mode,
              predictions=y,
              loss=cross_entropy,
              train_op=train,
              eval_metric_ops=eval_metric_ops)

x_train = mnist.train.images
y_train = mnist.train.labels.astype("float32")
x_eval = mnist.test.images
y_eval = mnist.test.labels.astype("float32")

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define the training & evaluation settings
input_fn = tf.estimator.inputs.numpy_input_fn(
                {"x": x_train},
                y_train,
                batch_size=128,
                num_epochs=None,
                shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                {"x": x_train},
                y_train,
                batch_size=128,
                num_epochs=1,
                shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                {"x": x_eval},
                y_eval,
                batch_size=128,
                num_epochs=1,
                shuffle=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\silver\\AppData\\Local\\Temp\\tmpyms9mkwm', '_num_worker_replicas': 1, '_session_config': None, '_master': '', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None, '_is_chief': True, '_tf_random_seed': None, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D7D35D1240>, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


# Training & Evaluation

In [4]:
# Training
estimator.train(input_fn=input_fn, steps=1000)

# evaluation
train_metrics = estimator.evaluate(input_fn=train_input_fn)["accuracy"]
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)["accuracy"]
print("\nTrain Accuracy: {0:f}\n".format(train_metrics))
print("\nTest Accuracy: {0:f}\n".format(eval_metrics))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\silver\AppData\Local\Temp\tmpyms9mkwm\model.ckpt.
INFO:tensorflow:step = 1, loss = 2.3025851
INFO:tensorflow:global_step/sec: 422.175
INFO:tensorflow:step = 101, loss = 0.24772504 (0.240 sec)
INFO:tensorflow:global_step/sec: 574.304
INFO:tensorflow:step = 201, loss = 0.3562895 (0.173 sec)
INFO:tensorflow:global_step/sec: 446.069
INFO:tensorflow:step = 301, loss = 0.311449 (0.225 sec)
INFO:tensorflow:global_step/sec: 577.624
INFO:tensorflow:step = 401, loss = 0.39457712 (0.173 sec)
INFO:tensorflow:global_step/sec: 470.675
INFO:tensorflow:step = 501, loss = 0.29495654 (0.214 sec)
INFO:tensorflow:global_step/sec: 580.459
INFO:tensorflow:step = 601, loss = 0.24754982 (0.171 sec)
INFO:tensorflow:global_step/sec: 587.666
INFO:tensorflow:step = 701, loss = 0.28392836 (0.170 sec)
INFO:tensorflow:global_step/sec: 562.914
INFO:tensorflow:step = 801, loss = 0.33547294 (0.178 sec)
INFO:tensorflow:glo